### DeepFool Algorithm <a name="deepfool"></a>

We have achieved some results with the CNN and MLP models, but we would like to explore how these models perform when faced with adversarial examples. To do so, we aply DeepFool.

**DeepFool** is a widely used method for generating adversarial examples, designed to evaluate the robustness of machine learning models, particularly in classification tasks. 

By iteratively finding the minimal perturbation needed to alter a model's prediction, DeepFool provides insights into a **model's vulnerability to adversarial attacks**. This method is crucial for developing more **robust and reliable models**, as it helps identify potential weaknesses and informs strategies for improving their defense mechanisms

![Descrição da imagem](./images/deepfool.png)


1. **Initialization:** Starting with the original image \( $x_0$ \), and setting the iteration counter \($ i $\) to 0.

2. **Perturbation Calculation:** For each iteration:
   - The algorithm calculates the perturbation required to change the model's prediction, iteratively adjusting the image.
   - It computes the gradients and the necessary perturbation \( $r_i $\) for the class boundary.

3. **Repeat Until Misclassification:** The algorithm repeats this process until the image is misclassified by the model.

4. **Return Perturbation:** The final perturbation is the sum of all the adjustments \( $r_i$ \) made during the iterations.

The result is the minimal perturbation $ r $ that causes a misclassification.


#### Imports 


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers, optimizers
from tensorflow.python.client import device_lib
import pandas as pd
import numpy as np
import pickle
import os
import librosa
from copy import deepcopy

#### Class MLP
[[go back to the top]](#deepfool)

The MLP model, similar to the one implemented in the [[MLP]](./MLP.ipynb) file, is built with the best hyperparameter configuration obtained previously and trained with validation.

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self, input_dim, output_dim, hidden_units, dropout_rate, activations, regularization_type=None, regularization_value=0.01):
        super(MLP, self).__init__()
        self.hidden_layers = []
        self.regularization_type = regularization_type
        self.regularization_value = regularization_value

        for units, activation in zip(hidden_units, activations):
            self.hidden_layers.append(
                tf.keras.layers.Dense(units, activation=activation)
            )
            self.hidden_layers.append(tf.keras.layers.Dropout(dropout_rate))
        
        self.output_layer = tf.keras.layers.Dense(output_dim, activation='softmax')  

    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)
    
    def compute_regularization_loss(self):
        regularization_loss = 0.0
        if self.regularization_type:
            for layer in self.hidden_layers:
                if isinstance(layer, tf.keras.layers.Dense):
                    weights = layer.kernel
                    if self.regularization_type == 'l1':
                        regularization_loss += tf.reduce_sum(tf.abs(weights)) * self.regularization_value
                    elif self.regularization_type == 'l2':
                        regularization_loss += tf.reduce_sum(tf.square(weights)) * self.regularization_value
        return regularization_loss

def load_fold_data(fold_index, files):
    # Adjust fold_index to be zero-based
    data = pd.read_csv(files[fold_index-1]).to_numpy()

    if np.isnan(data).any():
        print(f"Warning: Missing values detected in file {files[fold_index - 1]}.")
        data = data[~np.isnan(data).any(axis=1)]  # Remove rows with NaN values
    X = data[:, :-1]  # Features
    y = data[:, -1].astype(int)  # Labels
    if (y < 0).any() or (y >= 10).any():
        raise ValueError(f"Invalid label values detected in file {files[fold_index - 1]}. Labels: {np.unique(y)}")
    return X, y

files = [f'datasets/urbansounds_features_fold{i}.csv' for i in range(1, 11)]

# Define the test fold
fold_test = 1
X_test, y_test = load_fold_data(fold_test, files)

# Define the training folds
X_train, y_train = [], []
for i in range(1, 11):  # Total of 10 folds
    if i != fold_test:
        X_temp, y_temp = load_fold_data(i, files)
        X_train.append(X_temp)
        y_train.append(y_temp)

# Concatenate the training data
X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

# Hyperparameters
best_config = {
    'hidden_units': [256, 128, 64],
    'activations': ['relu', 'relu', 'relu'],
    'dropout_rate': 0.3,
    'batch_size': 64,
    'epochs': 20,
    'learning_rate': 0.0001,
    'regularization_type': None,
    'regularization_value': 0.01
}

# Initialize and train the model
model = MLP(
    input_dim=X_train.shape[1],
    output_dim=10,  # Classes from 0 to 9
    hidden_units=best_config['hidden_units'],
    dropout_rate=best_config['dropout_rate'],
    activations=best_config['activations']
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_config['learning_rate']),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Example validation split
X_val, y_val = X_train[:len(X_train)//10], y_train[:len(y_train)//10]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=best_config['batch_size'],
    epochs=best_config['epochs']
)


Epoch 1/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1875 - loss: 2.2475 - val_accuracy: 0.2917 - val_loss: 2.1037
Epoch 2/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3056 - loss: 2.0502 - val_accuracy: 0.3567 - val_loss: 1.9065
Epoch 3/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4009 - loss: 1.8247 - val_accuracy: 0.4420 - val_loss: 1.7311
Epoch 4/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4830 - loss: 1.6741 - val_accuracy: 0.4420 - val_loss: 1.6252
Epoch 5/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5298 - loss: 1.5154 - val_accuracy: 0.4943 - val_loss: 1.4976
Epoch 6/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5504 - loss: 1.4208 - val_accuracy: 0.5121 - val_loss: 1.4090
Epoch 7/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5720 - loss: 1.3396 - val_accuracy: 0.5580 - val_loss: 1.3154
Epoch 8/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5914 - loss: 1.2700 - val_accuracy: 0.

#### DeepFool Implementation
[[go back to the top]](#deepfool)

In this section we finally implement the algorithm. 

#### Hyperparameters


In [41]:
params = {
    'hidden_units': [256, 128, 64],
    'activations': ['relu', 'relu', 'relu'],
    'dropout_rate': 0.3,
    'batch_size': 64,
    'epochs': 20,
    'learning_rate': 0.0001,
    'regularization_type': None,
    'regularization_value': 0.01
}

#### Build the model
[[go back to the top]](#deepfool)

Here, we simply build and compile an MLP model based on the provided configuration as explained in the python script.

In [42]:
def build_mlp_model(input_dim, best_config):
    """
    Build and compile an MLP model based on the provided configuration.

    Args:
        input_dim (int): Number of input features.
        best_config (dict): Dictionary containing hyperparameters.

    Returns:
        tf.keras.Model: Compiled MLP model.
    """
    model = MLP(
                input_dim=X_train.shape[1],
                output_dim=10,
                hidden_units=best_config['hidden_units'],
                dropout_rate=best_config['dropout_rate'],
                activations=best_config['activations'],
                regularization_type=best_config.get('regularization_type', None),
                regularization_value=best_config.get('regularization_value', 0.01)
            )

    # Compile the model
    model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=best_config['learning_rate']),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy']
            )

    return model


#### Loading the datasets
[[go back to the top]](#deepfool)


In [43]:
# Load all folds into a list
files = [f'datasets/urbansounds_features_fold{i}.csv' for i in range(1, 11)]
folds = [pd.read_csv(file) for file in files]

#### Adversarial Training and Evaluation with DeepFool Implementation
[[go back to the top]](#deepfool)

This code trains an MLP model using predefined configurations, including **regularization** and **early stopping** for optimization. The `deepfool_mlp` function is introduced to generate adversarial examples by iteratively **perturbing input data to mislead the model**. This setup enables both robust model training and the exploration of adversarial attacks.

In [ ]:
def load_fold_data(fold_number, files):
    data = pd.read_csv(files[fold_number - 1]) 
    if data.empty:
        print(f"Erro: O arquivo {files[fold_number - 1]} está vazio ou não foi carregado corretamente.")
    labels = data.pop('Label').values
    features = data.values
    return features, labels


def train_evaluate_model(config, X_train, y_train, X_val, y_val):
    model = MLP(
        input_dim=X_train.shape[1],
        output_dim=10,
        hidden_units=config['hidden_units'],
        dropout_rate=config['dropout_rate'],
        activations=config['activations'],
        regularization_type=config.get('regularization_type', None),
        regularization_value=config.get('regularization_value', 0.01)
    )
    
    def loss_with_regularization(y_true, y_pred):
        base_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
        regularization_loss = model.compute_regularization_loss()
        return base_loss + regularization_loss

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config['learning_rate']),
        loss=loss_with_regularization,
        metrics=['accuracy']
    )
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        callbacks=[early_stopping],
        verbose=0
    )
    return history 

def deepfool_mlp(model, x0, y0, max_iter=50, epsilon=1e-6):
    x_adv = tf.identity(x0)  # Tensor inicial
    for i in range(max_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv)
            pred_label = tf.argmax(logits, axis=-1).numpy()[0]

        if pred_label != y0:
            return x_adv.numpy()  # Retorna o adversarial se o modelo errar

        gradients = tape.gradient(logits, x_adv).numpy()[0]
        logits = logits.numpy()[0]
        current_label = pred_label

        perturbations = []
        for k in range(len(logits)):
            if k != y0:
                diff_grad = gradients[k] - gradients[y0]
                diff_logits = logits[k] - logits[y0]
                perturbation = abs(diff_logits) / (np.linalg.norm(diff_grad) + epsilon)
                perturbations.append((perturbation, k))

        perturbations.sort(key=lambda x: x[0])  # Ordena pelo menor deslocamento necessário
        r_min, _ = perturbations[0]
        x_adv += r_min * gradients

    return x_adv.numpy()  # Retorna a última iteração como fallback


#### Cross-Validation with Adversarial Robustness Assessment using DeepFool
[[go back to the top]](#deepfool)

This code performs 10-fold cross-validation on the MLP model, evaluating its performance on **accuracy**, **loss**, and **robustness** against adversarial attacks generated by DeepFool. For each fold, the model is trained on a subset of the data, validated on another, and tested on the remaining fold. Robustness is measured by the **model's ability** to resist adversarial examples crafted for each test sample. The final outputs are the average accuracy, loss, and robustness across all folds, providing a comprehensive evaluation of the model.

In [48]:
def cross_validation_mlp_deepfool(datasets, model_builder, params):
    folds = list(range(1, 11))
    accuracy_values = []
    loss_values = []
    robustness_values = []

    for test_fold in folds:
        train_val_folds = [fold for fold in folds if fold != test_fold]
        train_folds = train_val_folds[:-1]
        val_fold = train_val_folds[-1]
        
        # Load data
        X_train, y_train = [], []
        for fold in train_folds:
            X_temp, y_temp = load_fold_data(fold, files)
            X_train.append(X_temp)
            y_train.append(y_temp)
        X_train = np.concatenate(X_train, axis=0)
        y_train = np.concatenate(y_train, axis=0)

        X_val, y_val = load_fold_data(val_fold, files)
        X_test, y_test = load_fold_data(test_fold, files)

        # Train model
        model = model_builder(input_dim=X_train.shape[1], best_config=best_config)
        model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            batch_size=params['batch_size'],
            epochs=params['epochs'],
            verbose=0
        )

        # Evaluate performance
        fold_loss, fold_accuracy = model.evaluate(X_test, y_test, verbose=0)
        accuracy_values.append(fold_accuracy)
        loss_values.append(fold_loss)

        # Robustness with DeepFool
        adversarial_success = 0
        for idx in range(len(X_test)):
            x0 = np.expand_dims(X_test[idx], axis=0)
            y0 = y_test[idx]
            x_adv = deepfool_mlp(model, x0, y0)
            adv_pred = tf.argmax(model(x_adv), axis=-1).numpy()[0]

            if adv_pred != y0:
                adversarial_success += 1

        robustness = 1 - (adversarial_success / len(X_test))
        robustness_values.append(robustness)
        print(f"Fold {test_fold}: Accuracy={fold_accuracy:.4f}, Loss={fold_loss:.4f}, Robustness={robustness:.4f}")

    # Calculate averages
    avg_accuracy = np.mean(accuracy_values)
    avg_loss = np.mean(loss_values)
    avg_robustness = np.mean(robustness_values)

    # Display averages
    print("\nCross-Validation Summary:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Average Robustness: {avg_robustness:.4f}")

    return accuracy_values, loss_values, robustness_values

accuracy_values, loss_values, robustness_values = cross_validation_mlp_deepfool(
    datasets=files,            
    model_builder=build_mlp_model,  
    params=best_config        
)

Fold 1: Accuracy=0.6804, Loss=1.1354, Robustness=0.5945
Fold 2: Accuracy=0.5439, Loss=1.2987, Robustness=0.4899
Fold 3: Accuracy=0.6032, Loss=1.1992, Robustness=0.5178
Fold 4: Accuracy=0.5808, Loss=1.2251, Robustness=0.4980
Fold 5: Accuracy=0.6154, Loss=1.0490, Robustness=0.5331
Fold 6: Accuracy=0.5176, Loss=1.5814, Robustness=0.4520
Fold 7: Accuracy=0.5525, Loss=1.3745, Robustness=0.5060
Fold 8: Accuracy=0.6154, Loss=1.2348, Robustness=0.5186
Fold 9: Accuracy=0.5417, Loss=1.3398, Robustness=0.4608
Fold 10: Accuracy=0.6045, Loss=1.1937, Robustness=0.5293

Cross-Validation Summary:
Average Accuracy: 0.5855
Average Loss: 1.2632
Average Robustness: 0.5100


#### Conclusion
[[go back to the top]](#deepfool)


- The **cross-validation** results provide valuable insights into the performance of our MLP model. The average accuracy of **58.55%** indicates that the model performs moderately well at classifying the urban sound data. 

- The average loss of **1.2632** suggests that the model struggles to confidently separate some classes. 

- The robustness score, averaging **51.00%**, reveals that the model is somewhat vulnerable to adversarial attacks crafted by DeepFool.

While the results aren't exactly what we hoped for, they show that there's plenty of room for **improvement**, and that's something we're excited to keep working on.

Looking back, this project came with its fair share of challenges, especially when it came to feature extraction and modeling. But overall, it was a really rewarding experience. We gained a much deeper understanding of how neural networks work and how different hyperparameters (such as activation functions, epochs, etc...) can really affect the outcome. It’s been a great learning process, and we’re looking forward to building on what we've learned to improve the model further.